# Day 2b - Agent Tools Best Practices #

> ⚠️ **Nota importante**  
>  A continuación se muestra un proceso secuencial inicial, a nivel de configuración, que es necesario realizar antes de empezar a trabajar con los Agentes. Se importan librerias de ADK necesarias con sus respectivos componentes, se crean las Helper Functions y las opciones de configuración de Retry.  


In [1]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [2]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)
print("✅Retry Options configured sucessfully.")

✅Retry Options configured sucessfully.


 
> ⚠️ **Nota importante**  
>  Fin de confiduraciones e integraciones. 

## Sección 2: Protocolo de Contexto del Modelo (Model Context Protocol)

Hasta ahora, has aprendido a crear funciones personalizadas para tus agentes. Sin embargo, conectarse a sistemas externos (GitHub, bases de datos, Slack) requiere escribir y mantener clientes de API.

El ***Model Context Protocol (MCP)*** es un estándar abierto que permite a los agentes usar integraciones creadas por la comunidad. En lugar de escribir tus propias integraciones y clientes de API, simplemente conéctate a un servidor MCP existente.

MCP permite a los agentes:

- ✅ Acceder a datos externos y en tiempo real desde bases de datos, APIs y servicios sin necesidad de escribir código de integración personalizado
- ✅ Aprovechar herramientas creadas por la comunidad con interfaces estandarizadas
- ✅ Escalar capacidades conectándose a múltiples servidores especializados

 
> ⚠️ **Nota importante**  
>  Conectando a MCP. 

Uso de MCP con tu agente

El flujo de trabajo es simple:

Elegir un servidor MCP y una tool

Crear el conjunto de tools MCP (configurar la conexión)

Añadirlo a tu agente

Ejecutar y probar el agente

***Paso 1: Elegir un servidor MCP***

Para esta demostración, usaremos el Everything MCP Server, un paquete npm (@modelcontextprotocol/server-everything) diseñado para probar integraciones MCP.

Este servidor proporciona una tool llamada getTinyImage, que devuelve una imagen de prueba sencilla (16×16 píxeles, codificada en Base64).
Puedes encontrar más servidores en: modelcontextprotocol.io/examples

‼️ NOTA: Este es un servidor de demostración para aprender MCP.
En producción, usarías servidores para Google Maps, Slack, Discord, etc.

***Paso 2: Crear el conjunto de tools MCP***

El McpToolset se utiliza para integrar un agente de ADK con un servidor MCP.

Qué hace el código:

Usa npx (ejecutor de paquetes de Node) para ejecutar el servidor MCP

Se conecta a @modelcontextprotocol/server-everything

Filtra para usar solo la tool getTinyImage (el servidor tiene más, pero solo necesitamos esta)

In [7]:
# MCP integration with Everything Server
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


***Paso 3: Añadir la tool MCP al agente***

Vamos a añadir el mcp_server al arreglo (lista) de tools del agente y actualizar las instrucciones del agente para manejar solicitudes de generación de imágenes pequeñas (tiny images).


In [8]:
# Create image agent with MCP integration
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="Use the MCP Tool to generate images for user queries",
    tools=[mcp_image_server],
)

***Creamos el Runner***


In [9]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)

***Testeamos el Agente***

In [12]:
response = await runner.run_debug("Proporciona una imagen pequeña de ejemplo", verbose=True)



 ### Continue session: debug_session_id

User > Proporciona una imagen pequeña de ejemplo


ConnectionError: Failed to create MCP session: fileno

***Codigo Correcto, pero no soportable en Ipython, o en Jupyter..hay que correrlo en archivos.py***